<a href="https://colab.research.google.com/github/ravindrakush11/AI-Agents/blob/main/HF_Agentic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q smolagents pandas langchain langchain-community sentence-transformers datasets python-dotenv rank_bm25 --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
# Load environment variables (including HF_TOKEN)
from dotenv import load_dotenv
load_dotenv()

True

## Step 2: Prepare the Knowledge Base

In [9]:
from importlib.metadata import metadata
import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    add_start_index = True,
    strip_whitespace= True,
    separators = ["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)

print(f"Knowledge base prepare with {len(docs_processed)} document chunks")

Knowledge base prepare with 14695 document chunks


## Create Retriever Tool

In [11]:
from smolagents import Tool

class RetrieverTool(Tool):
  name = "retriever"
  description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
  inputs = {
      "query": {
          "type": "string",
          "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
      }
  }
  output_type = "string"

  def __init__(self, docs, **kwargs):
    super().__init__(**kwargs)
    self.retriever = BM25Retriever.from_documents(
        docs, k = 10
    )

  def forward(self, query: str) -> str:
    assert isinstance(query, str)

    docs = self.retriever.invoke(query)
    return "\nRetrieved documents: \n" + "".join(
        [
            f"\n\n==== Document {str(i)} ====\n" + doc.page_content
            for i, doc in enumerate(docs)
        ]
    )

retriever_tool = RetrieverTool(docs_processed)

## Step 4: Create an Advanced Retrieval Agent

In [ ]:
from smolagents import InferenceClientModel, CodeAgent

agent = CodeAgent(
    tools = [retriever_tool],
    model = InferenceClientModel(),
    max_steps = 4,
    verbosity_level = 2,
)

# use a specific model
# model=InferenceClientModel(model_id="meta-llama/Llama-3.3-70B-Instruct")

## Step 5: Run the Agent to Answer Question

In [ ]:
question = "For a transformers model training, which is slower, the forward or backward pass?"

agent_output = agent.run(question)

print("\n Final Answer: ")
print(agent_output)